In [1]:
!pip install psycopg2

In [2]:
import psycopg2
import sqlalchemy

import pandas as pd

from datetime import date, datetime
import time
from timeit import default_timer as timer

from io import StringIO

import time as tm

import os

In [3]:
import sys
print(sys.version.split()[0])

3.9.7


In [3]:
#Informações de conexão com o banco Postgres
host='dataiesb.iesbtech.com.br'
dbname='IESB_Empresas'
user='1922120026_TCC_Thiago'
password='**********'

In [4]:
# Função para criar conexão no banco
def conecta_db():
    con = psycopg2.connect(host=host, 
                         database=dbname,
                         user=user, 
                         password=password)
    return con

def postgres_test():
    try:
        con = conecta_db()
        con.close()
        print("Conexão com o banco de dados realizada com sucesso!")
    except:
        print("A conexão com o banco de dados falhou!")

postgres_test()

Conexão com o banco de dados realizada com sucesso!


In [5]:
# Função para criar tabela no banco
def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()

In [6]:
# Criando a tabela para os dados das CAGED EXCLUIDOS
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_exc (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
competenciaexc varchar(255),
indicadordeexclusao varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255),
ano_mes varchar(255)
  );'''
criar_db(sql)

# Criando a tabela para os dados das CAGED MOVIMENTOS
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_mov (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255),
ano_mes varchar(255)
  );'''
criar_db(sql)

# Criando a tabela para os dados das CAGED FORA DO PRAZO
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_for (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255),
ano_mes varchar(255)
  );'''
criar_db(sql)

In [7]:
path = 'C:/Users/thiago.silva/TCC/CAGED/dados'

In [11]:
def copy_from_stringio(con, file, base, reference, table):
    buffer = StringIO()
    df = pd.read_csv(file, delimiter=';', decimal=',')
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    
    cursor = con.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cursor.close()
        return 1
    print(f'copy_from_stringio() done for CAGED{base}{reference}')
    cursor.close()

In [9]:
reference=[]
def references(anos,meses):
    for ano in anos:
        for mes in meses:
            reference.append(ano+mes)

anos = ['2020', '2021','2022','2023']
meses = ['01','02','03','04','05','06', '07', '08', '09', '10', '11', '12']
references(anos,meses)

In [12]:
con = conecta_db()

bases = ['EXC', 'MOV', 'FOR']

for base in bases:
    for i in reference:
        nome_arquivo = f'CAGED{base}{i}.csv'
        file = os.path.join(path, nome_arquivo)
        if os.path.exists(file):
            table = f'tb_caged_{base.lower()}'
            print(f'Inserindo {i} em {table}')
            copy_from_stringio(con, file, base, i, table)

Inserindo 202004 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202004
Inserindo 202005 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202005
Inserindo 202006 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202006
Inserindo 202007 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202007
Inserindo 202008 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202008
Inserindo 202009 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202009
Inserindo 202010 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202010
Inserindo 202011 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202011
Inserindo 202012 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202012
Inserindo 202101 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202101
Inserindo 202102 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202102
Inserindo 202103 em tb_caged_exc
copy_from_stringio() done for CAGEDEXC202103
Inserindo 202104 em tb_caged_exc
copy_from_stringio() done for C

copy_from_stringio() done for CAGEDFOR202208
Inserindo 202209 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202209
Inserindo 202210 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202210
Inserindo 202211 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202211
Inserindo 202212 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202212
Inserindo 202301 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202301
Inserindo 202302 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202302
Inserindo 202303 em tb_caged_for
copy_from_stringio() done for CAGEDFOR202303
